## 사전작업

In [1]:
# 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [15]:
import pandas as pd
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import matplotlib.font_manager as fm
# 설치된 폰트 중 'NanumGothic' 찾기
for font in fm.fontManager.ttflist:
    if 'NanumGothic' in font.name:
        plt.rcParams['font.family'] = font.name
        break
plt.rcParams['axes.unicode_minus'] = False
import seaborn as sns
import numpy as np
import scipy.stats as stats

## 데이터 로드

### 데이터 로드 및 원하는 데이터 추출

In [30]:
df_elderly = pd.read_csv('./elderly.csv')
df_elderly

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
1,66445cd65350d17a77350dc6,1269,밝은내데이케어센터 전문강사 채용 공고,밝은내데이케어센터,24-03-11,187,2024년 밝은내데이케어센터 전문강사 채용 공고\n 밝은내데이케어센터는 이용 어르...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
3,66445cfd5350d17a77350dc8,1267,참사람들시니어케어 사회복지사 채용(긴급) 1명,성남재가노인복지센터,24-02-21,187,Ⅰ. 모집분야 및 자격기준\n1. 채용인원 : 장기요양서비스(방문요양) 사회복지사 ...
4,66445d035350d17a77350dc9,1266,[채용]시립마포노인종합복지관병설 데이케어센터 사회복지사 채…,마포데이케어센터,24-01-05,320,[채용] 마포데이케어센터 공고 제 2024-01호\n 마포데이케어센터 사회복지사 ...
...,...,...,...,...,...,...,...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...
1242,664478245350d17a7735129f,43,단원구노인복지관 방문요양 요양보호사 모집,NaN,08-09-19,13202,▒ 기관정보 ▒\n------------------------------------...


In [31]:
df_elderly_2 = df_elderly[df_elderly['title'].str.contains('간호조무사|요양보호사', na=False)]
df_elderly_2

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,23-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1235,664477fe5350d17a77351298,50,요양보호사 및 사회복지사 채용 공고,NaN,08-10-02,14300,▒ 기관정보 ▒\n------------------------------------...
1236,664478035350d17a77351299,49,경기 안성 감로당 가정봉사원파견센터 요양보호사 1급 모집,감로당단기보호,08-10-02,14299,첨부파일을 참조해 주세요~
1237,664478085350d17a7735129a,48,대구 요양보호사1급 모십니다,NaN,08-09-30,15109,▒ 기관정보 ▒\n------------------------------------...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...


In [32]:
df_elderly_3 = df_elderly[df_elderly['content[0]'].str.contains('간호조무사|요양보호사', na=False)]
df_elderly_3

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
8,66445d195350d17a77350dcd,1262,[채용]시립마포노인종합복지관 데이케어센터 파트타임 간호조무…,마포데이케어센터,23-05-10,1786,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
9,66445d1f5350d17a77350dce,1261,[긴급채용]시립마포노인종합복지관 데이케어센터 파트타임 요양…,마포데이케어센터,23-05-10,1946,[채용] 마포데이케어센터 공고 제 2023-02호\n 마포데이케어센터 요양보호사 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
...,...,...,...,...,...,...,...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...
1242,664478245350d17a7735129f,43,단원구노인복지관 방문요양 요양보호사 모집,NaN,08-09-19,13202,▒ 기관정보 ▒\n------------------------------------...


In [33]:
df_concat = pd.concat([df_elderly_2,df_elderly_3])
df_concat = df_concat.drop_duplicates()
df_concat

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,23-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1213,664477845350d17a77351282,72,도솔노인복지센터 직원채용공고,NaN,08-10-20,14389,▒ 기관정보 ▒\n------------------------------------...
1238,6644780e5350d17a7735129b,47,직원채용공고,NaN,08-09-28,14125,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...


### 모집한 데이터를 분석

In [34]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         771 non-null    object
 1   number      771 non-null    int64 
 2   title       771 non-null    object
 3   writter     684 non-null    object
 4   date        771 non-null    object
 5   count       771 non-null    int64 
 6   content[0]  771 non-null    object
dtypes: int64(2), object(5)
memory usage: 48.2+ KB


In [35]:
df_concat['date'] = pd.to_datetime(df_concat['date'], format='%y-%m-%d')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   _id         771 non-null    object        
 1   number      771 non-null    int64         
 2   title       771 non-null    object        
 3   writter     684 non-null    object        
 4   date        771 non-null    datetime64[ns]
 5   count       771 non-null    int64         
 6   content[0]  771 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 48.2+ KB


In [36]:
df_concat

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,2024-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,2024-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,2023-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,2023-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,2023-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1213,664477845350d17a77351282,72,도솔노인복지센터 직원채용공고,NaN,2008-10-20,14389,▒ 기관정보 ▒\n------------------------------------...
1238,6644780e5350d17a7735129b,47,직원채용공고,NaN,2008-09-28,14125,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,2008-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,2008-09-22,14426,▒ 기관정보 ▒\n------------------------------------...


In [37]:
# 'date' 열에서 연도를 추출하여 'year' 열을 생성
df_concat['year'] = df_concat['date'].dt.year

# 'year' 열을 기준으로 그룹화하고 각 그룹의 크기를 계산
year_counts = df_concat.groupby('year').size()

print(year_counts)


year
2008     71
2009    153
2010     90
2011     77
2012     64
2013     53
2014     45
2015     26
2016     29
2017     33
2018     26
2019     15
2020     20
2021     32
2022     15
2023     18
2024      4
dtype: int64


In [38]:
df_elderly['date'] = pd.to_datetime(df_elderly['date'], format='%y-%m-%d')
df_elderly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   _id         1244 non-null   object        
 1   number      1244 non-null   int64         
 2   title       1244 non-null   object        
 3   writter     1122 non-null   object        
 4   date        1244 non-null   datetime64[ns]
 5   count       1244 non-null   int64         
 6   content[0]  1242 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 68.2+ KB


In [39]:
# 'date' 열에서 연도를 추출하여 'year' 열을 생성
df_elderly['year'] = df_elderly['date'].dt.year

# 'year' 열을 기준으로 그룹화하고 각 그룹의 크기를 계산
year_count = df_elderly.groupby('year').size()

print(year_count)

year
2008    100
2009    220
2010    127
2011    121
2012     95
2013     82
2014     71
2015     48
2016     50
2017     65
2018     60
2019     48
2020     41
2021     52
2022     27
2023     25
2024     12
dtype: int64


In [43]:
df_concat['content[0]'].value_counts()

content[0]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [44]:
df_concat.reset_index(drop=True, inplace=True)
df_concat

,_id,number,title,writter,date,count,content[0],year
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,2024-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...,2024
1,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,2024-03-07,172,,2024
2,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,2023-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...,2023
3,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,2023-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬...",2023
4,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,2023-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...,2023
...,...,...,...,...,...,...,...,...
766,664477845350d17a77351282,72,도솔노인복지센터 직원채용공고,NaN,2008-10-20,14389,▒ 기관정보 ▒\n------------------------------------...,2008
767,6644780e5350d17a7735129b,47,직원채용공고,NaN,2008-09-28,14125,▒ 기관정보 ▒\n------------------------------------...,2008
768,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,2008-09-26,14318,▒ 기관정보 ▒\n------------------------------------...,2008
769,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,2008-09-22,14426,▒ 기관정보 ▒\n------------------------------------...,2008


In [45]:
# 각 컬럼별 전처리(제목)
df_concat['title']

0                   은평데이케어센터 주간 요양보호사 구인
1                  도계재가노인복지센터 요양보호사 채용공고
2            구립 역삼데이케어센터 직원(요양보호사) 채용 공고
3      [채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …
4                ◆번동데이케어센터 야간요양보호사 채용공고◆
                     ...                
766                      도솔노인복지센터 직원채용공고
767                               직원채용공고
768                  새로 open하는 노인복지센터입니다
769           [직원채용공고] 보령주간노인복지센터 직원채용공고
770                   효경소규모노인종합센터 직원채용공고
Name: title, Length: 771, dtype: object

In [47]:
df_concat['content[0]'] = df_concat['content[0]'].str.replace(r'\n', '')

df_concat['content[0]']

0      은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와같이...
1                                                       
2      구립 역삼데이케어센터 직원(요양보호사) 채용 공고강남구립 역삼데이케어센터에서는 고령...
3      「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬...
4      1. 모집분야가. 채용내용 채용 분야인원근무조건담당업무근무기간야간요양보호사1명주 5...
                             ...                        
766    ▒ 기관정보 ▒--------------------------------------...
767    ▒ 기관정보 ▒--------------------------------------...
768    ▒ 기관정보 ▒--------------------------------------...
769    ▒ 기관정보 ▒--------------------------------------...
770    1. 채용명 : 효경소규모노인종합센터 입소보호 어르신에게 서비스를 제공할 생활지도원...
Name: content[0], Length: 771, dtype: object

In [48]:
concat_series = df_concat['title'] + ' ' + df_concat['content[0]']
concat_series.isna().sum()

0

In [49]:
concat_series = concat_series.tolist()
concat_series

['은평데이케어센터 주간 요양보호사 구인 은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와같이 모집하오니 많은관심과 응시바랍니다1. 채용내용① 채용직무 및 인원: 주간 요양보호사 1명② 채용형태 : 계약직(2024.6.1~2025.5.31)2. 응시자격 : 다음의 결격 사유에 해당하지 않는 자(결격사유 해당시 합격 및 고용 취소)①범죄 및 성폭력범죄 조회 시 결격사유가 없는 자②성별, 연령 제한 없음    (단, 서울시 규정에 의거 정년 만60세 초과되지 않는 자)3. 근무조건① 근무일시 : 월~금(1일 8시간 / 08:00~17:00)② 급여는 데이케어센터 내부 지침 기준에 의함③ 4대보험 및 퇴직연금 가입4. 제출서류① 이력서, 자기소개서, 개인정보 제공 동의서(양식은 반드시 첨부파일 양식으로 사용하시기 바랍니다)② 국가유공자 등 예우 및 지원에 관한 법률 제29조에 의한 취업지원대상자는 입사지원서에 해당 사항을 체크해주시기 바랍니다.5. 채용방법- 1차 : 서류전형- 2차 : 면접(1차 서류전형 합격자에 한함)6. 채용일정① 채용공고 및 원서접수 : 2024년 5월 7일(화) ~ 5월 22일(수) 18:00까지② 서류전형 합격자발표 : 2024년 5월 22일(수)③ 면접시험 및 최종발표 : 2024년 5월 23일(목)④ 지원서류 마감 : 2024년 5월 22일(수) 18:00까지 도착분⑤ 면 접 : 서류전형 합격자에 한하여 면접 실시(개별통보)- 면접일정은 상황에 따라 변경될 수 있습니다.7. 합격자 발표① 1차 합격자 발표 : 합격/불합격 여부 홈페이지 공지② 2차 합격자 발표 : 합격/불합격 여부 홈페이지 공지8. 접수처① 접 수 처 : 서울시립은평노인종합복지관 병설 은평데이케어센터② 지원서 양식은 상단에 첨부파일을 통해서 다운받으시기 바랍니다.③ 접수방법 : 우편, 이메일접수(ep-silver@naver.com), 방문접수9. 접수 및 문의처① 전 화 : 02-385-8731 / 팩스 385-8724② 홈페이지 : ww

In [50]:
import re
patterns = '()'
series = []
for total_list in concat_series:
    total_list = str(total_list)
    text_regex = re.sub(pattern=patterns,repl='',string=total_list)
    if text_regex != '':
        series.append(text_regex)
series[:2]

['은평데이케어센터 주간 요양보호사 구인 은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와같이 모집하오니 많은관심과 응시바랍니다1. 채용내용① 채용직무 및 인원: 주간 요양보호사 1명② 채용형태 : 계약직(2024.6.1~2025.5.31)2. 응시자격 : 다음의 결격 사유에 해당하지 않는 자(결격사유 해당시 합격 및 고용 취소)①범죄 및 성폭력범죄 조회 시 결격사유가 없는 자②성별, 연령 제한 없음    (단, 서울시 규정에 의거 정년 만60세 초과되지 않는 자)3. 근무조건① 근무일시 : 월~금(1일 8시간 / 08:00~17:00)② 급여는 데이케어센터 내부 지침 기준에 의함③ 4대보험 및 퇴직연금 가입4. 제출서류① 이력서, 자기소개서, 개인정보 제공 동의서(양식은 반드시 첨부파일 양식으로 사용하시기 바랍니다)② 국가유공자 등 예우 및 지원에 관한 법률 제29조에 의한 취업지원대상자는 입사지원서에 해당 사항을 체크해주시기 바랍니다.5. 채용방법- 1차 : 서류전형- 2차 : 면접(1차 서류전형 합격자에 한함)6. 채용일정① 채용공고 및 원서접수 : 2024년 5월 7일(화) ~ 5월 22일(수) 18:00까지② 서류전형 합격자발표 : 2024년 5월 22일(수)③ 면접시험 및 최종발표 : 2024년 5월 23일(목)④ 지원서류 마감 : 2024년 5월 22일(수) 18:00까지 도착분⑤ 면 접 : 서류전형 합격자에 한하여 면접 실시(개별통보)- 면접일정은 상황에 따라 변경될 수 있습니다.7. 합격자 발표① 1차 합격자 발표 : 합격/불합격 여부 홈페이지 공지② 2차 합격자 발표 : 합격/불합격 여부 홈페이지 공지8. 접수처① 접 수 처 : 서울시립은평노인종합복지관 병설 은평데이케어센터② 지원서 양식은 상단에 첨부파일을 통해서 다운받으시기 바랍니다.③ 접수방법 : 우편, 이메일접수(ep-silver@naver.com), 방문접수9. 접수 및 문의처① 전 화 : 02-385-8731 / 팩스 385-8724② 홈페이지 : ww

### 구분

In [236]:
# 불용어 리스트 생성
f=open('./stopwords.txt')
stopwords=[]
lines = f.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f.close()

In [237]:
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun", "Adverb"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
    ]

In [238]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
# tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2)
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.80, min_df=3)

In [239]:
series = [' '.join(x.split()) for x in series]
features = tfidfVectorizer.fit_transform(series)
features.toarray()[:2]

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0.        , 0.        , 0.07252118, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [240]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [241]:
lda_model  = LatentDirichletAllocation(n_components=3, n_jobs=2) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기
lda_model.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=2)

In [242]:
vis = pyLDAvis.lda_model.prepare(lda_model, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [243]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

In [244]:
concat_series

filter = []
for total_list in concat_series:
    try:
        if '자기' in total_list:
            total_list = str(total_list)
            print(total_list)
            print("--------")
        else : 
            pass
    except:
        print(total_list)
        pass

은평데이케어센터 주간 요양보호사 구인 은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와같이 모집하오니 많은관심과 응시바랍니다1. 채용내용① 채용직무 및 인원: 주간 요양보호사 1명② 채용형태 : 계약직(2024.6.1~2025.5.31)2. 응시자격 : 다음의 결격 사유에 해당하지 않는 자(결격사유 해당시 합격 및 고용 취소)①범죄 및 성폭력범죄 조회 시 결격사유가 없는 자②성별, 연령 제한 없음    (단, 서울시 규정에 의거 정년 만60세 초과되지 않는 자)3. 근무조건① 근무일시 : 월~금(1일 8시간 / 08:00~17:00)② 급여는 데이케어센터 내부 지침 기준에 의함③ 4대보험 및 퇴직연금 가입4. 제출서류① 이력서, 자기소개서, 개인정보 제공 동의서(양식은 반드시 첨부파일 양식으로 사용하시기 바랍니다)② 국가유공자 등 예우 및 지원에 관한 법률 제29조에 의한 취업지원대상자는 입사지원서에 해당 사항을 체크해주시기 바랍니다.5. 채용방법- 1차 : 서류전형- 2차 : 면접(1차 서류전형 합격자에 한함)6. 채용일정① 채용공고 및 원서접수 : 2024년 5월 7일(화) ~ 5월 22일(수) 18:00까지② 서류전형 합격자발표 : 2024년 5월 22일(수)③ 면접시험 및 최종발표 : 2024년 5월 23일(목)④ 지원서류 마감 : 2024년 5월 22일(수) 18:00까지 도착분⑤ 면 접 : 서류전형 합격자에 한하여 면접 실시(개별통보)- 면접일정은 상황에 따라 변경될 수 있습니다.7. 합격자 발표① 1차 합격자 발표 : 합격/불합격 여부 홈페이지 공지② 2차 합격자 발표 : 합격/불합격 여부 홈페이지 공지8. 접수처① 접 수 처 : 서울시립은평노인종합복지관 병설 은평데이케어센터② 지원서 양식은 상단에 첨부파일을 통해서 다운받으시기 바랍니다.③ 접수방법 : 우편, 이메일접수(ep-silver@naver.com), 방문접수9. 접수 및 문의처① 전 화 : 02-385-8731 / 팩스 385-8724② 홈페이지 : www.

In [245]:
dictionary_list = tfidfVectorizer.get_feature_names_out(features)
dictionary_list

array(['가사', '가이드라인', '가입', ..., '휴일', '희망', '희망근로'], dtype=object)

In [246]:
lda_model.components_

array([[0.3354991 , 0.39152553, 0.72337962, ..., 0.33401109, 0.39729339,
        0.33465731],
       [2.30093948, 2.10526763, 5.60868126, ..., 1.32660343, 0.33439522,
        1.23620647],
       [0.63516625, 0.33712754, 5.78945401, ..., 0.33384946, 1.48739428,
        0.33445277]])

In [247]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [248]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1)

In [249]:
df_topics_score['sentences'] = series
df_topics_score

,0,1,2,dominant_topic_number,sentences
0,0.039493,0.920608,0.039900,1,은평데이케어센터 주간 요양보호사 구인 은평데이케어센터에서는 가족 같은 마음으로 어르...
1,0.184236,0.648736,0.167028,1,도계재가노인복지센터 요양보호사 채용공고
2,0.040683,0.122434,0.836883,2,구립 역삼데이케어센터 직원(요양보호사) 채용 공고 구립 역삼데이케어센터 직원(요양보...
3,0.029340,0.941269,0.029391,1,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 … 「마포데이케어센터」에서...
4,0.066419,0.890563,0.043018,1,◆번동데이케어센터 야간요양보호사 채용공고◆ 1. 모집분야가. 채용내용 채용 분야인원...
...,...,...,...,...,...
766,0.091366,0.821596,0.087038,1,도솔노인복지센터 직원채용공고 ▒ 기관정보 ▒----------------------...
767,0.086403,0.091196,0.822402,2,직원채용공고 ▒ 기관정보 ▒-------------------------------...
768,0.100057,0.426371,0.473572,2,새로 open하는 노인복지센터입니다 ▒ 기관정보 ▒------------------...
769,0.079827,0.640940,0.279233,1,[직원채용공고] 보령주간노인복지센터 직원채용공고 ▒ 기관정보 ▒-----------...


In [250]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topics_keywords = pd.DataFrame(topics_list_add)

방문 간호조무사 야간 제한 회관 중원구 요양 노인 목욕 복지
노인 복지 사회 간호조무사 야간 보호 서비스 우대 방문 운전
노인 복지 방문 주간 요양 간호조무사 우대 보험 사회 동대문구


In [251]:
df_topics_keywords

,0,1,2
0,방문 간호조무사 야간 제한 회관 중원구 요양 노인 목욕 복지,노인 복지 사회 간호조무사 야간 보호 서비스 우대 방문 운전,노인 복지 방문 주간 요양 간호조무사 우대 보험 사회 동대문구
1,Topic0,Topic1,Topic2


In [252]:
pivot_table = pd.pivot_table(data=df_topics_score,
               values='sentences',
               index='dominant_topic_number',
               aggfunc='count')
pivot_table

,sentences
dominant_topic_number,
0,149
1,423
2,199
